Instroduction:

Diabetes is a chronic disease that affects millions of people worldwide, and it can lead to a range of serious complications if not managed properly.
By developing accurate predictive models for diabetes, healthcare providers can identify patients who are at high risk for developing complications or 
experiencing adverse health outcomes, and intervene early to prevent these outcomes.

Early identification of high-risk patients: By using predictive models, healthcare providers can identify patients who are at high risk for developing
complications such as kidney disease, neuropathy, or retinopathy, and intervene early to prevent or delay these complications.

More personalized care: Predictive models can help healthcare providers tailor their treatment plans to the specific needs of each patient, taking into 
account their individual risk factors, medical history, and other relevant factors.

Reduced healthcare costs: By identifying high-risk patients early and intervening to prevent complications, healthcare providers can reduce the overall
cost of care for diabetic patients.

Improved patient outcomes: By using predictive models to identify high-risk patients and intervene early, healthcare providers can improve patient outcomes
and quality of life.

Overall, studying predictive models for diabetic patients can help healthcare providers provide more effective, personalized care, and ultimately improve
patient outcomes while reducing healthcare costs

We aim to find possible correlation between 
The dataset used for the project can be found on the following web address: 
https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset

We have used the second of the 3 files for the purpose of this project. It has been cleaned and wrangled from another data file which can be found on the following web address: 
https://www.kaggle.com/cdc/behavioral-risk-factor-surveillance-system
This dataset was created using the annual telephone survey conducted by the Center for Disease Control and Prevention under The Behavioral Risk Factor Surveillance System (BRFSS) program. 

Finally, we adapted the dataset and publiched it online using GitHub. This is the dataset we read in our project. It can be found here:
https://drive.google.com/u/0/uc?id=1OAZCpZGdFPy70ll_Fo2ow5dpaM1sG_47&amp;export=download

!!! Describe the dataset with the file we use: what factors, what cols involve?
!!! more detail question: how does a concoction of factors such as Age, BMI, Smoking habits, 
Income effect a non-diabetic person's chances of acquiring type-2 diabetes?

Method and Result:
    
While there may be numerous methods to conduct such an analysis, we find that a predictive model would suit best for our purpose of the research for two reasons: 
    reason 1: our target is to identify a categorical variable
    reason 2: a relatively simple algorithm to implement, since it only requires calculating distances between data points and selecting the k-nearest neighbors.

# should we or how we improve those method?
 We shall first run some preliminary analysis before deploying sophisticated tools to analyse areas of interest. Our classifier model will take variables such as a person's age, bmi, smoking habits, income etc. into account to identify non-diabetic patients most at risk of acquiring diabetes. 
This can be done in several ways. For instance, the average distance of 3 nearest diabetic patients can be used to determine how likely a person is to acquire diabetes. Participants of the survey who's average distance to the 3 nearest diabetics on the graph is below a certain level can be classified most at-risk.
Another possible method is to use K-nearest neighbors classification algorithm, although this method would be more suitable to classify someone as diabetic or non-diabetic rather than predicting which respondents are most at-risk.

# my draft:
1. we run some preliminary analysis to select the factors in raw dataset, identify the potential predictors to build our predictive classification model
2. in order to build a good-quality classification model, we should split the data into training dataset and testing dataset, which can be use to test the quality of overall classifier
!only preprocess the training dataset

#cross-validation
3. Then we should work on our training dataset. The classifier need to pick a K value to maximize the accuracy. Thus we choose to use cross-validation. If we just split the tranining data once to evaluate, the K maybe strongly depends on the specific valiadation and sub-training dataset, which lead to be overfitting or underfitting 
4. In R, we can use the vfold_cv function to conduct cross-validation. To utilize this function, we need to  indicate the number of folds (v) and the categorical variable.
5. Generate a new model specification for K-nearest neighbor but instead of defining a value for the K, using function tune()
6. create a workflow() analysis that put fruit_recipe and new knn_tune model specification.
7. plot the line diagram which can find different K with different accuracy. In this case, we should roughly optiminal accuracy but not too expensive for calculating (not too large);  does not change much if you change KK to a nearby value

8. Then put this K into our best fit classifier, training the model and using the test dataset to evaluate the predictive model.


In [1]:
# code part without writing

library(tidyverse)
library(repr)
library(tidymodels)
#options(repr.matrix.max.rows = 6)
#source('tests.R')
#source('cleanup.R')

url <- "https://drive.google.com/u/0/uc?id=1OAZCpZGdFPy70ll_Fo2ow5dpaM1sG_47&amp;export=download"
diabetes_data <- read_csv(url)

summary(diabetes_data)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

 Diabetes_binary     HighBP          HighChol        CholCheck     
 Min.   :0.0     Min.   :0.0000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.0     1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:1.0000  
 Median :0.5     Median :1.0000   Median :1.0000   Median :1.0000  
 Mean   :0.5     Mean   :0.5635   Mean   :0.5257   Mean   :0.9753  
 3rd Qu.:1.0     3rd Qu.:1.0000   3rd Qu.:1.0000   3rd Qu.:1.0000  
 Max.   :1.0     Max.   :1.0000   Max.   :1.0000   Max.   :1.0000  
      BMI            Smoker           Stroke        HeartDiseaseorAttack
 Min.   :12.00   Min.   :0.0000   Min.   :0.00000   Min.   :0.0000      
 1st Qu.:25.00   1st Qu.:0.0000   1st Qu.:0.00000   1st Qu.:0.0000      
 Median :29.00   Median :0.0000   Median :0.00000   Median :0.0000      
 Mean   :29.86   Mean   :0.4753   Mean   :0.06217   Mean   :0.1478      
 3rd Qu.:33.00   3rd Qu.:1.0000   3rd Qu.:0.00000   3rd Qu.:0.0000      
 Max.   :98.00   Max.   :1.0000   Max.   :1.00000   Max.   :1.0000      
  PhysActivit

In [2]:
diabetes_data <- diabetes_data |>
    mutate(Smoker = as.logical(Smoker)) |>
    mutate(Diabetes_binary = as.logical(Diabetes_binary)) |>
    select(BMI,Age,Income,Smoker,Diabetes_binary)

diabetes_data

BMI,Age,Income,Smoker,Diabetes_binary
<dbl>,<dbl>,<dbl>,<lgl>,<lgl>
26,4,8,FALSE,FALSE
26,12,8,TRUE,FALSE
26,13,8,FALSE,FALSE
28,11,8,TRUE,FALSE
29,8,8,TRUE,FALSE
18,1,7,FALSE,FALSE
26,13,6,TRUE,FALSE
31,6,3,TRUE,FALSE
32,3,8,FALSE,FALSE


In [3]:
# make more readable, slice the data
slice(diabetes_data, 1:10)

BMI,Age,Income,Smoker,Diabetes_binary
<dbl>,<dbl>,<dbl>,<lgl>,<lgl>
26,4,8,FALSE,FALSE
26,12,8,TRUE,FALSE
26,13,8,FALSE,FALSE
28,11,8,TRUE,FALSE
29,8,8,TRUE,FALSE
18,1,7,FALSE,FALSE
26,13,6,TRUE,FALSE
31,6,3,TRUE,FALSE
32,3,8,FALSE,FALSE


In [ ]:
# ??? DO a exploratory data analysis to identify the potential predictors? 
# !! need to improve to more concose

In [ ]:
# ??? visualization for the dataset after exploratory data analysis:
# Use the diagram to show relationship between each factor? maybe use ggpair???

In [6]:
set.seed(1)
diabetes_split <- initial_split(diabetes_data, prop =0.75, strata = Diabetes_binary) 
diabetes_train <-training(diabetes_split)
diabetes_test <-testing(diabetes_split)

#make more readable and show that our sub dataset already shuffle)
#should we show that the dataset is balanced and stra
slice(diabetes_train,1:10)
slice(diabetes_test,1:10)

BMI,Age,Income,Smoker,Diabetes_binary
<dbl>,<dbl>,<dbl>,<lgl>,<lgl>
26,4,8,FALSE,FALSE
26,12,8,TRUE,FALSE
26,13,8,FALSE,FALSE
28,11,8,TRUE,FALSE
29,8,8,TRUE,FALSE
18,1,7,FALSE,FALSE
26,13,6,TRUE,FALSE
31,6,3,TRUE,FALSE
24,12,6,TRUE,FALSE


BMI,Age,Income,Smoker,Diabetes_binary
<dbl>,<dbl>,<dbl>,<lgl>,<lgl>
32,3,8,FALSE,FALSE
27,6,4,TRUE,FALSE
27,7,8,FALSE,FALSE
18,10,6,TRUE,FALSE
22,6,8,FALSE,FALSE
29,10,8,TRUE,FALSE
22,8,8,FALSE,FALSE
28,8,8,FALSE,FALSE
24,4,1,TRUE,FALSE


In [ ]:
# tuning and build the classifier by using cross-validation

In [ ]:
# using best accuracy K put into the classifier and train

In [ ]:
# using test dataset to evaluate the quality of classifier, then interpret the result

In [ ]:
# ?? create the visualization for the analysis? how to do that with multivariable classifier?

# Discussion: (writing part)
    1. what you find?: from what: 
        classifier accuracy
        is predictors help to improve accuracy? ...
        
    2. different with expectation?
    3. impact of this finding?
    4. lead what further questions?: For instance, can a similar model be built for all such diseases which do not have outright physical indicators. If models can be built for predicting or classifying patients as healthy, at-risk and patient, then could a model be built for curing diabetes and similar diseases?